# Decision Tree

### Merge feature and test datasets

In [1]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from pandas import DatetimeIndex
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#We have to merge our data
def mergeData(df):
    features =pd.read_csv('features.csv')
    storesdata =pd.read_csv('stores.csv')
    df = pd.merge(df, features, on=['Store','Date','IsHoliday'],how='inner')
    df = pd.merge(df, storesdata, on=['Store'], how='inner')
    return df


In [2]:
merged_df = mergeData(pd.read_csv('train.csv'))

In [3]:
merged_df.fillna(value=0, inplace=True)

merged_df['Markdowns'] = merged_df['MarkDown1'] + merged_df['MarkDown2'] + merged_df['MarkDown3'] + merged_df['MarkDown4'] + merged_df['MarkDown5'] 
labelsToDrop = ['MarkDown1', 'MarkDown2', 'MarkDown3','MarkDown4','MarkDown5']
merged_df.drop(labels=labelsToDrop,axis=1, inplace=True)

In [4]:
merged_df.loc[merged_df['Weekly_Sales']>300000,"Date"].value_counts()

df = merged_df
df.Date = pd.to_datetime(df.Date)

#Converts the date into three columns
df['Year'] = DatetimeIndex(df['Date']).year
df['Month']= DatetimeIndex(df['Date']).month
df['Day'] = DatetimeIndex(df['Date']).day
df = df.drop(columns=['Date'])

print(df.head())

   Store  Dept  Weekly_Sales  IsHoliday  Temperature  Fuel_Price         CPI  \
0      1     1      24924.50      False        42.31       2.572  211.096358   
1      1     2      50605.27      False        42.31       2.572  211.096358   
2      1     3      13740.12      False        42.31       2.572  211.096358   
3      1     4      39954.04      False        42.31       2.572  211.096358   
4      1     5      32229.38      False        42.31       2.572  211.096358   

   Unemployment Type    Size  Markdowns  Year  Month  Day  
0         8.106    A  151315        0.0  2010      2    5  
1         8.106    A  151315        0.0  2010      2    5  
2         8.106    A  151315        0.0  2010      2    5  
3         8.106    A  151315        0.0  2010      2    5  
4         8.106    A  151315        0.0  2010      2    5  


In [5]:
#creating the feature matrix 
def categorical_to_numeric(x):
    if x == False:
        return 0
    elif x == True:
        return 1
    
df['IsHoliday'] = df['IsHoliday'].apply(categorical_to_numeric)

y =df['Weekly_Sales']
X = df.drop(columns=['Weekly_Sales','Type'])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

## Decision Tree

In [7]:
#Decision Tree
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

## Testing on the testing set

In [8]:
#Our predictions
y_predict_dt = regressor.predict(X_test)
print(y_predict_dt)

[ 198.   1630.3  8484.62 ...   85.5  7630.86 3260.06]


## Accuracy Evaluation

In [9]:
#Accuracy score based on predictions
#score_dt = accuracy_score(y_test, y_predict_dt)

score_dt = regressor.score(X_test, y_test)
print(score_dt)

0.9539125333897658


## Feature Importance

In [10]:
for name, importance in zip(X.columns, regressor.feature_importances_):
    print(name, importance)

Store 0.06833741910999895
Dept 0.6329163628327762
IsHoliday 0.002192471094166806
Temperature 0.010236437245977767
Fuel_Price 0.004028267493162888
CPI 0.02834015409977218
Unemployment 0.010058710096131133
Size 0.18799620139712928
Markdowns 0.002831697364017206
Year 0.00029404314039587187
Month 0.031595819006091624
Day 0.021172417120380177
